In [1]:
import keras.applications.resnet50 as resnet
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

Using TensorFlow backend.


In [2]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16
DROPOUT = 0.5

# 이미지 volume 값
input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH)

# 모델 받아오기
model_origin = resnet.ResNet50(include_top=False, weights=None, input_shape=input_shape, pooling='avg', classes=LABEL_NUM)

# 모델 dropout 설정
output = model_origin.output

#output = Flatten()(output)
output = Dense(2048, activation="relu")(output)
output = Dropout(DROPOUT)(output)
output = Dense(LABEL_NUM, activation='softmax', name='predictions')(output)

model = Model(model_origin.input, output)

# 모델 출력
model.summary()

# Compling
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:
#배치 크기
BATCH_SIZE = 32

#훈련용 이미지 생성
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

training_set = train_datagen.flow_from_directory('./Dataset/Train',
                                                target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                batch_size = BATCH_SIZE)

validation_set = validation_datagen.flow_from_directory('./Dataset/Validation',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# Val값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping()

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

#배치 수 계산
train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)

Found 69267 images belonging to 16 classes.
Found 9588 images belonging to 16 classes.


In [4]:
#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/ResNet/" + NOW)

#훈련!
model.fit_generator(training_set,
                    steps_per_epoch = train_step_epoch,
                    epochs = 100,
                    validation_data = validation_set,
                    validation_steps = validation_step_epoch,
                    class_weight=class_weights,
                    shuffle=True,
                    callbacks=[tensorboard])

#모델 저장
model_json = model.to_json()
with open("./Models/ResNet/model_resnet_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)

# Weight 저장
model.save_weights("./Models/ResNet/model_resnet_weight_" + NOW + ".h5")
print("Saved model to disk")

Epoch 1/100
2164/2164 [==============================] - 396s 183ms/step - loss: 13.7313 - acc: 0.0500 - val_loss: 5.4184 - val_acc: 0.0436
Epoch 2/100
2164/2164 [==============================] - 336s 155ms/step - loss: 12.0233 - acc: 0.0771 - val_loss: 3.0185 - val_acc: 0.0444
Epoch 3/100
2164/2164 [==============================] - 319s 148ms/step - loss: 11.5478 - acc: 0.1238 - val_loss: 2.6865 - val_acc: 0.2149
Epoch 4/100
2164/2164 [==============================] - 318s 147ms/step - loss: 11.4970 - acc: 0.1261 - val_loss: 2.7277 - val_acc: 0.1088
Epoch 5/100
2164/2164 [==============================] - 319s 148ms/step - loss: 11.4502 - acc: 0.1162 - val_loss: 2.6496 - val_acc: 0.1541
Epoch 6/100
2164/2164 [==============================] - 318s 147ms/step - loss: 11.3866 - acc: 0.1145 - val_loss: 2.7385 - val_acc: 0.1065
Epoch 7/100
2164/2164 [==============================] - 318s 147ms/step - loss: 11.2652 - acc: 0.1200 - val_loss: 2.7540 - val_acc: 0.1576
Epoch 8/100
2164/216

2164/2164 [==============================] - 318s 147ms/step - loss: 6.5470 - acc: 0.3298 - val_loss: 4.4195 - val_acc: 0.1174
Epoch 60/100
2164/2164 [==============================] - 318s 147ms/step - loss: 6.4129 - acc: 0.3336 - val_loss: 4.3078 - val_acc: 0.1349
Epoch 61/100
2164/2164 [==============================] - 318s 147ms/step - loss: 6.3039 - acc: 0.3407 - val_loss: 4.4403 - val_acc: 0.1272
Epoch 62/100
2164/2164 [==============================] - 318s 147ms/step - loss: 6.1778 - acc: 0.3486 - val_loss: 5.0456 - val_acc: 0.1526
Epoch 63/100
2164/2164 [==============================] - 318s 147ms/step - loss: 6.1328 - acc: 0.3510 - val_loss: 4.8122 - val_acc: 0.1327
Epoch 64/100
2164/2164 [==============================] - 318s 147ms/step - loss: 5.9099 - acc: 0.3617 - val_loss: 4.9702 - val_acc: 0.1488
Epoch 65/100
2164/2164 [==============================] - 318s 147ms/step - loss: 5.8610 - acc: 0.3623 - val_loss: 4.5527 - val_acc: 0.1319
Epoch 66/100
2164/2164 [=========

In [5]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset/Test',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

Found 9181 images belonging to 16 classes.
-- Evaluate --
acc: 13.19%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.047 0.004 0.000 0.000 0.000 0.000 0.000 0.000 0.297 0.000 0.000 0.000
 0.224 0.047 0.212 0.168]


In [7]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])

-- Evaluate --
acc: 34.58%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.042 0.005 0.000 0.000 0.001 0.002 0.001 0.002 0.046 0.003 0.000 0.003
 0.049 0.006 0.047 0.794]
